In [1]:
# Import Dependencies
from flask import Flask, render_template, request, render_template_string, jsonify
import serial
from datetime import datetime
import numpy as np
import pandas as pd
import time
import threading

In [2]:
# Mimic Inital BPM
initial_bpm = 80
start_pressed = False
i = 1
bpm = 0
user_bpm_data = []
user_bpm_time = []
user_heart_beat_data = []

In [3]:
# Function to Read BPM over Serial
def serial_data_read():
    global i
    global initial_bpm
    global bpm
    global user_bpm_data
    
    print('serial_data_read executed...')
    #bpm = str(initial_bpm + 2) + ' BPM'
    
    if (bpm < 120):
        bpm = initial_bpm + i
        user_bpm_data.append(bpm)
        user_bpm_time.append(str(datetime.time(datetime.now()))[:-4])
    return bpm

In [4]:
# Read Serial Data from Arduino
# def serial_data_read():
#     # Open Serial COM Port
#     Arduino_serial = serial.Serial("COM4", 9600)
#     # Read Data from Serial COM Port
#     ser_data = Arduino_serial.readline().strip()
#     return ser_data

# # Write to Arduino to control Read/Write State
# def serial_data_write(data=None):
#     # Open Serial COM Port
# #     if (serial.Serial("COM4", 9600).isOpen == False):
# #         Arduino_serial = serial.Serial("COM4", 9600)
#     while(True):
#         # Send Data to Arduino '1': Start; '0': Stop
#         Arduino_serial.write(data.encode())
#         # 2 second delay
#         time.sleep(2)
#         ser_data = Arduino_serial.readline().strip()
#         print(ser_data.decode())
#         return '85 BPM'
#     return 'Problem opening COM Port. . .'

In [5]:
app = Flask(__name__, template_folder='templates')


@app.route('/')
def index():
    return render_template('main.html', beats_per_minute= initial_bpm)


# Function to Save User Data to CSV File
def writeBPMData(fileName=None):
    # Convert Array of values to dataframe
    df = pd.DataFrame({'BPM':np.asarray(user_bpm_data), 'Time':user_bpm_time})
    df.to_csv(fileName, index=False)
    print('Saved user data as {}.csv.'.format(fileName))


def plotHeartBeat():
    plt.figure(figsize=(20,10))
    plt.title('Heart Beat')
    plt.plot(np.asarray(user_heart_beat_data), label='heart_beat')
    plt.legend()
    plt.show()
    

# Function to Write data to CSV file and Plot in Real Time
def save_and_plot(fileName=None):
    # Open Serial COM Port
    Arduino_serial = serial.Serial("COM4", 9600)

    with open(fileName,'w') as f:
    while (True):
        ser_data = Arduino_serial.readline().strip()
        if (int(ser_data.decode()) > threshold):
            print(ser_data.decode())
            f.write(ser_data.decode())
            f.write('\n')
    
    
@app.route('/', methods=['GET','POST'])
def data():
    print('Button Function...')
    # Variable to hold BPM Values
    bpm = 0
    text = ' '
    ser_start = False
    usr_name= 'Anuj'
    threshold = 550
    
    
    # Format ( request.form[name] == value )
    if (request.method == 'POST'):
        # Get User Name and Gender for Generating Report
        if (request.form['button'] == 'user_name_submit'):
            usr_name = request.form['user_name']
            name = usr_name.split(',')[0]
            usr_gender = usr_name.split(',')[1]
            print('User Name: {0}\t User Gender: {1}'.format(usr_name, usr_gender))
        
        # Start Recording Data in CSV File and Plot Data in Real Time
        elif (request.form['button'] == 'start'):
            # Function to Start Reading the data
            def serial_read():
                #global i
                global initial_bpm
                print('Reading BPM from Arduino over Serial. . .')
                #bpm = initial_bpm + i
                bpm = serial_data_read()
                initial_bpm = bpm
                print("New BPM: ",str(bpm) + ' BPM')
                return jsonify(beats_per_minute = bpm)
            serial_read()
            print('Executing Function...')
        
        elif (request.form['button'] == 'stop'):
            print('Stopping Data Read over Serial. . .')
            # Write '0' to Arduino to Stop Reading Data; Arduino Stop Writing Data
            #bpm = serial_write(data='0')
            writeData(fileName=usr_name+'.csv')
            bpm = 0
            return render_template('main.html', beats_per_minute= bpm)
        
        elif (request.form['button'] == 'showPlot'):
            ser_data = Arduino_serial.readline().strip()
            
            if (ser_data > threshold):
                user_heart_beat_data.append(ser_data.decode())
    
        
    elif (request.method == 'GET'):
        return render_template('main.html',  beats_per_minute= bpm)
    
    #print('End of Function...')
    #print(bpm, initial_bpm)
    return render_template('main.html', beats_per_minute= initial_bpm)


# Main Function
if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2018 13:46:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2018 13:47:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2018 13:47:04] "POST / HTTP/1.1" 200 -


Button Function...
Reading BPM from Arduino over Serial. . .
serial_data_read executed...
New BPM:  81 BPM
Executing Function...


127.0.0.1 - - [29/Sep/2018 13:47:10] "POST / HTTP/1.1" 200 -


Button Function...
Reading BPM from Arduino over Serial. . .
serial_data_read executed...
New BPM:  82 BPM
Executing Function...


127.0.0.1 - - [29/Sep/2018 13:47:15] "POST / HTTP/1.1" 200 -


...

Button Function...
Reading BPM from Arduino over Serial. . .
serial_data_read executed...
New BPM:  90 BPM
Executing Function...


127.0.0.1 - - [29/Sep/2018 13:47:59] "POST / HTTP/1.1" 200 -


Button Function...
Stopping Data Read over Serial. . .
Saved user data as Anuj.csv.csv.


127.0.0.1 - - [29/Sep/2018 13:48:05] "POST / HTTP/1.1" 200 -


Button Function...
Stopping Data Read over Serial. . .
Saved user data as Anuj.csv.csv.
